In [ ]:
# Create the model
import pytorch_lightning as pl
import mlflow
import dagshub
import torch
import os
import json
import requests
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchmetrics
from pathlib import Path
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.profilers import PyTorchProfiler
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature
from pprint import pprint

In [ ]:
# Hyperparameters
NUM_CLASSES = 10  # number of genres
INPUT_SIZE = 13  # number of MFCC coefficients
HIDDEN_SIZE = 128
NUM_LAYERS = 2
BATCH_SIZE = 64
NUM_EPOCHS = 50
LEARNING_RATE = 1e-3

NUM_WORKERS = 1
VALIDATION_SIZE = 0.25
TEST_SIZE = 0.2
DATASET_PATH = Path.cwd().parent / "data" / "processed" / "genres_mfccs.json"

ARTIFACT_PATH = "genre_classifier"
MODEL_NAME = "genre-classifier"

CONDA_PATH = Path.cwd().parent / "environment.yaml"
CODE_PATH = Path.cwd()

In [ ]:
# Dataset
class MFCCDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        mfccs = self.X[idx]
        label = self.y[idx]
        return mfccs, label


class MFCCDataModule(pl.LightningDataModule):
    def __init__(
        self,
        dataset_path,
        batch_size,
        num_workers,
        test_size,
        validation_size,
    ):
        super().__init__()
        self.dataset_path = dataset_path
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.test_size = test_size
        self.validation_size = validation_size

    @staticmethod
    def load_data(dataset_path):
        """
        Loads training dataset from json file.
            :param data_path (str): Path to json file containing data
            :return X (ndarray): Inputs
            :return y (ndarray): Targets
        """
        with open(dataset_path, "r") as fp:
            print("Loading Data")
            data = json.load(fp)
            # convert lists to numpy arrays
            X = np.array(data["mfcc"])
            # X = np.array(data["spectrogram"])
            y = np.array(data["labels"])
            mappings = data["mappings"]
            return X, y, mappings

    @staticmethod
    def prepare_datasets(
        X, y, test_size, validation_size, shuffle=True, random_state=42
    ):
        # create train, validation and test split
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            test_size=test_size,
            shuffle=shuffle,
            random_state=random_state,
        )

        # create train/validation split
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=validation_size
        )

        return X_train, y_train, X_test, y_test, X_val, y_val

    def setup(self, stage=None):
        # Load data
        self.X, self.y, _ = self.load_data(self.dataset_path)

        # Convert to tensors
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.long)

        # Create train/val/test split
        (
            self.X_train,
            self.y_train,
            self.X_val,
            self.y_val,
            self.X_test,
            self.y_test,
        ) = self.prepare_datasets(
            self.X,
            self.y,
            self.test_size,
            self.validation_size,
        )

        # Create dataset objects
        self.train_dataset = MFCCDataset(self.X_train, self.y_train)
        self.test_dataset = MFCCDataset(self.X_test, self.y_test)
        self.val_dataset = MFCCDataset(self.X_val, self.y_val)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)


class LSTMGenreModel(pl.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        num_classes,
        learning_rate,
        dataset_path: str | Path,
    ):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()
        self.dataset_path = dataset_path
        self.learning_rate = learning_rate
        self.accuracy = torchmetrics.Accuracy(
            task="multiclass", num_classes=num_classes
        )
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

    def _common_step(self, batch):
        mfccs, labels = batch
        outputs = self(mfccs)
        loss = self.loss_fn(outputs, labels)
        return loss, outputs, labels

    def training_step(self, batch):
        loss, outputs, labels = self._common_step(batch)
        accuracy = self.accuracy(outputs, labels)
        f1_score = self.f1_score(outputs, labels)
        self.log_dict(
            {
                "train_loss": loss,
                "train_accuracy": accuracy,
                "train_f1_score": f1_score,
            },
            on_step=False,
            on_epoch=True,
            prog_bar=True,
        )
        return {"loss": loss, "outputs": outputs, "labels": labels}

    def validation_step(self, batch):
        loss, outputs, labels = self._common_step(batch)
        self.log("val_loss", loss)
        return loss

    # Test step
    def test_step(self, batch):
        loss, outputs, labels = self._common_step(batch)
        self.log("test_loss", loss)
        return loss

    def save_checkpoint(self, checkpoint_path, filename="checkpoint.ckpt"):
        torch.save(self.state_dict(), os.path.join(checkpoint_path, filename))

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    

In [ ]:
model = LSTMGenreModel(
        input_size=INPUT_SIZE,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        num_classes=NUM_CLASSES,
        learning_rate=LEARNING_RATE,
        dataset_path=DATASET_PATH,
    )

dm = MFCCDataModule(
    dataset_path=DATASET_PATH,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    validation_size=VALIDATION_SIZE,
    test_size=TEST_SIZE,
)

# dagshub.init(
#     repo_owner="stephenjera",
#     repo_name="Genre-Classification",
#     mlflow=True,
# )
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.pytorch.autolog()

# logger = TensorBoardLogger("tb_runs")
profiler = PyTorchProfiler(
    on_trace_ready=torch.profiler.tensorboard_trace_handler("tb_runs/profiler0"),
    schedule=torch.profiler.schedule(
        skip_first=10,
        wait=1,
        warmup=1,
        active=20,
    ),
)
trainer = pl.Trainer(
    # profiler=profiler,
    # logger=logger,
    max_epochs=NUM_EPOCHS,
    log_every_n_steps=25,
)
trainer.fit(model, dm)
# trainer.validate(model, dm)
# trainer.test(model, dm)
# trainer.save_checkpoint("checkpoint.ckpt")


# Create sample input
input_tensor = torch.rand(1, 259, 13) 
# Make predictions
predictions = model(input_tensor)
# Input tensor
input_np = input_tensor.numpy() 
# Output tensor 
predictions_np = predictions.detach().numpy()
# Infer signature
signature = infer_signature(input_np, predictions_np)

mlflow.pytorch.log_model(
    pytorch_model=model,
    artifact_path=MODEL_NAME,
    conda_env=str(CONDA_PATH),
    # code_paths=[str(CODE_PATH)],
    signature=signature
)
run_id = mlflow.active_run().info.run_id
# mlflow.register_model(f"runs:/{run_id}/{ARTIFACT_PATH}", MODEL_NAME)

In [ ]:
X, y, mapings = MFCCDataModule.load_data(DATASET_PATH)

In [ ]:
X[:2].shape

In [ ]:
single_mfcc = X[:1].tolist()

# Make prediction request
url = 'http://localhost:1234/invocations'
headers = {'Content-Type': 'application/json'}
data = {
  "instances": single_mfcc
}
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print prediction
pprint(response.json())

In [ ]:
reverse_dict = {v: k for k, v in mapings.items()}

In [ ]:
predictions = response.json()['predictions']

# Convert the predictions to a numpy array
predictions_array = np.array(predictions)

# Find the index of the maximum value
argmax_index = np.argmax(predictions_array)

print(f"prediction:{argmax_index}, {reverse_dict[argmax_index]} Actual {y[0]}, {reverse_dict[y[0]]}")